# Libraries needed for running the models.

This program imports all the libraries needed for running the ANN-GARCH model. Please install all the libraries before running the code.

In [ ]:
import pandas as pd
import numpy as np
# python -m pip install yahoo-finance; pip install yahoo-finance; pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
# pip install --upgrade ta; pip install ta
import ta as ta
import sklearn as sk
from sklearn import preprocessing
from scipy.stats import t
import tensorflow as tf
from datetime import date, datetime, timedelta
from arch import arch_model
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

# Functions for fitting ANN-GARCH are defined.

This program is used for loading all the formulas needed for fitting ANN-GARCH model.

In [7]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database['Close'][i+dif])-np.log(Database['Close'][i]))
    return np.append(np.repeat(np.nan, dif),Out), Database.index

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def TrueSDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif+1])
    for i in range (dimension-dif+1):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(Out,np.repeat(np.nan, dif-1))

#Database is calculated
def DatabaseGeneration (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index) 
    return Data.dropna()

#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11

#Fitting of GJR_GARCH(1,1)
def GJR_GARCH_Model_Student (Data):
    AR_Data=Data['DailyReturns']*100
    GJR_GARCH11 = arch_model(AR_Data, p=1, o=1, q=1, dist ='t')
    res_GJR_GARCH11 = GJR_GARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_GJR_GARCH11 = res_GJR_GARCH11.conditional_volatility
    For_CV_GJR_GARCH11 = np.array(res_GJR_GARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return GJR_GARCH11, res_GJR_GARCH11, CV_GJR_GARCH11, For_CV_GJR_GARCH11

#Fitting of TARCH(1,1)
def TARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    TARCH11 = arch_model(AR_Data, p=1, o=1, q=1, power=1.0, dist ='t')
    res_TARCH11 = TARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_TARCH11 = res_TARCH11.conditional_volatility
    For_CV_TARCH11 = np.array(res_TARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return TARCH11, res_TARCH11, CV_TARCH11, For_CV_TARCH11

#Fitting of EGARCH(1,1)
def EGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    EGARCH11 = arch_model(AR_Data, dist ='t', vol="EGARCH")
    res_EGARCH11 = EGARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_EGARCH11 = res_EGARCH11.conditional_volatility
    For_CV_EGARCH11 = np.array(res_EGARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return EGARCH11, res_EGARCH11,CV_EGARCH11, For_CV_EGARCH11

#Fitting of Absolute Value GARCH(1,1)
def AVGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    AVGARCH11 = arch_model(AR_Data, dist ='t', power=1)
    res_AVGARCH11 = AVGARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_AVGARCH11 = res_AVGARCH11.conditional_volatility
    For_CV_AVGARCH11 = np.array(res_AVGARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return AVGARCH11, res_AVGARCH11, CV_AVGARCH11, For_CV_AVGARCH11

#Fitting of FIGARCH11(1,1)
def FIGARCH_Model_Student(Data):
    AR_Data=Data['DailyReturns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res_FIGARCH11 = FIGARCH11.fit(disp='off',options={'maxiter': 1000})
    CV_FIGARCH11 = res_FIGARCH11.conditional_volatility
    For_CV_FIGARCH11 = np.array(res_FIGARCH11.forecast(horizon=1).variance.dropna())[0][0]   
    return FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11

#AR models are fitted. As requested by arma package, returns are multiplied by 100 in order to improve the fitting process.
#GARCH(1,1), GJR_GARCH(1,1), TARCH(1,1), EGARCH(1,1), AVGARCH(1,1) and FIGARCH(1,1) volatility models are fitted.
#T student is assumed as distribution.
def AR_Models (Data):
    GARCH, GARCH_Parameters, CV_GARCH, For_CV_GARCH = GARCH_Model_Student(Data)
    GJR_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH = GJR_GARCH_Model_Student(Data)
    TARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH = TARCH_Model_Student(Data)
    EGARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH = EGARCH_Model_Student(Data)
    AVGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH = AVGARCH_Model_Student(Data)
    FIGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH  = FIGARCH_Model_Student(Data)
    return GARCH_Parameters, CV_GARCH, For_CV_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH

#Fully Connected model is created
def FC_Model (Input_Dimension_NN, DropoutRate, LearningRate):
    #Model struture is defined
    Inputs = tf.keras.Input(shape=(Input_Dimension_NN,), name="Input")
    X = tf.keras.layers.Dense(16, activation=tf.nn.sigmoid, name="Dense1") (Inputs)
    X = tf.keras.layers.Dropout(DropoutRate) (X)
    X = tf.keras.layers.Dense(8, activation=tf.nn.sigmoid, name="Dense2") (X)
    X = tf.keras.layers.Dropout(DropoutRate) (X)
#    X = tf.keras.layers.Dense(4, activation=tf.nn.sigmoid, name="Dense3") (X)
#    X = tf.keras.layers.Dropout(DropoutRate) (X)
    Output = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid, name="Output") (X)
    #Output = tf.keras.layers.Dropout(DropoutRate) (X)
    model = tf.keras.Model(inputs=Inputs, outputs=Output)
    #Optimizer is defined
    Opt = tf.keras.optimizers.Adam(learning_rate=LearningRate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
    #Model is compiled
    model.compile(optimizer=Opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

#Database is calculated
def DatabaseGenerationForecast (Database, Lag, LagSD):
    DailyReturns, Index = ReturnCalculation(Database,Lag)
    DailyReturnsOld =  np.append(np.repeat(np.nan, 1),DailyReturns[0:(DailyReturns.shape[0]-1)])
    SD = SDCalculation (DailyReturns, LagSD)
    TrueSD = TrueSDCalculation(DailyReturns, LagSD)
    Data = pd.DataFrame({'DailyReturns': DailyReturns, 'SD': SD, 'TrueSD': TrueSD, 'DailyReturnsOld': DailyReturnsOld})
    Data = Data.set_index(Index) 
    return Data

#Final AR database for forcasting is generated
def DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH):
    Data_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).iloc[(-LagSD+1)]
    Index_Forecast=DatabaseGenerationForecast(Database, Lag, LagSD).index[(-LagSD+1)]
    XDataForecast={'SD': Data_Forecast['SD'], 'DailyReturnsOld': Data_Forecast['DailyReturnsOld'], 
               'CV_GARCH' : For_CV_GARCH/100, 'CV_GJR_GARCH' : For_CV_GJR_GARCH/100, 'CV_TARCH' : For_CV_TARCH/100, 
               'CV_EGARCH' : For_CV_EGARCH/100, 'CV_AVGARCH' : For_CV_AVGARCH/100, 'CV_FIGARCH' : For_CV_FIGARCH/100}
    return pd.DataFrame([XDataForecast], index=[Index_Forecast]), Data_Forecast['DailyReturns']

#ANN-ARCH model forecast
def ANN_ARCH_Forecast (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH,Scaled_Norm, XData_AR, model):
    XDataForecast, ReturnForecast = DatabaseGenerationForecast_AR (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH)
    XDataForecast = pd.concat([XData_AR,XDataForecast])
    XDataForecastTotalScaled_T = Scaled_Norm.transform(XDataForecast)
    TransformerPrediction = model.predict(XDataForecastTotalScaled_T)
    return TransformerPrediction[-1][0], XDataForecast.index[-1], TransformerPrediction[0:(XDataForecastTotalScaled_T.shape[0]-1)], ReturnForecast

#It calculates VaR taking into consideration the forecasted sigma to calculate the scale parameter
def ANN_ARCH_VaR (Alpha, HistoricalReturns, ForecastedSigma, DF):
    HistoricalMean = np.mean(HistoricalReturns)
    ScaleParameter = np.sqrt((ForecastedSigma**2)*((DF-2)/DF))
    VaR = -t.ppf(Alpha, DF, loc=HistoricalMean, scale=ScaleParameter)
    return VaR

#Formula to calculate the VaR of ARCH models
def VaR_AR_Model (AR_Model,AR_Model_Results,Alpha):
    Cond_Var=AR_Model_Results.forecast(horizon=1).variance.dropna()
    Cond_Mean=AR_Model_Results.forecast(horizon=1).mean.dropna()
    Quantile_Dist=AR_Model.distribution.ppf([Alpha], AR_Model_Results.params[-1:])
    VaR=(-Cond_Mean-np.sqrt(Cond_Var)*Quantile_Dist)/100
    return VaR.values

#Formula to calculate the VaR of all the ARCH models
def VaR_AR_Total(Alpha, GARCH_fit, GJR_GARCH_fit, TARCH_fit, EGARCH_fit, AVGARCH_fit, FIGARCH_fit,GARCH, GJR_GARCH, TARCH, EGARCH, AVGARCH, FIGARCH):
    VaR_GARCH = VaR_AR_Model (GARCH,GARCH_fit,Alpha)
    VaR_GJR_GARCH = VaR_AR_Model (GJR_GARCH,GJR_GARCH_fit,Alpha)
    VaR_TARCH = VaR_AR_Model (TARCH,TARCH_fit,Alpha)
    VaR_EGARCH = VaR_AR_Model (EGARCH,EGARCH_fit,Alpha)
    VaR_AVGARCH = VaR_AR_Model (AVGARCH,AVGARCH_fit,Alpha)
    VaR_FIGARCH = VaR_AR_Model (FIGARCH,FIGARCH_fit,Alpha)
    return {'VaR_GARCH':VaR_GARCH, 'VaR_GJR_GARCH':VaR_GJR_GARCH, 'VaR_TARCH':VaR_TARCH, 'VaR_EGARCH':VaR_EGARCH, 'VaR_AVGARCH':VaR_AVGARCH, 'VaR_FIGARCH':VaR_FIGARCH}

#Fitting of ANN-ARCH model and forecasting of the next volatility value
def ANN_ARCH_Fit (Data, Lag=1, LagSD=5, Dropout=0.05, LearningRate=0.001, Epochs=10000, Alpha=0.005, DF=4, BatchSize=64):
    #AR Models are fitted
    GARCH, GARCH_Parameters, CV_GARCH, For_CV_GARCH = GARCH_Model_Student(Data)
    GJR_GARCH, GJR_GARCH_Parameters, CV_GJR_GARCH, For_CV_GJR_GARCH = GJR_GARCH_Model_Student(Data)
    TARCH, TARCH_Parameters, CV_TARCH, For_CV_TARCH = TARCH_Model_Student(Data)
    EGARCH, EGARCH_Parameters,CV_EGARCH, For_CV_EGARCH = EGARCH_Model_Student(Data)
    AVGARCH, AVGARCH_Parameters,CV_AVGARCH, For_CV_AVGARCH = AVGARCH_Model_Student(Data)
    FIGARCH, FIGARCH_Parameters,CV_FIGARCH, For_CV_FIGARCH  = FIGARCH_Model_Student(Data)
    #Database contaning AR models is generated
    Data_AR=pd.concat([Data, CV_GARCH.rename('CV_GARCH')/100, CV_GJR_GARCH.rename('CV_GJR_GARCH')/100, CV_TARCH.rename('CV_TARCH')/100, 
                     CV_EGARCH.rename('CV_EGARCH')/100, CV_AVGARCH.rename('CV_AVGARCH')/100, CV_FIGARCH.rename('CV_FIGARCH')/100], axis=1)
    if Data_AR.shape[0]!=Data.shape[0]: print("Error in DB Generation")
    #Original explanatory and response variables are generated
    XData_AR = Data_AR.drop(Data_AR.columns[[0,2]], axis=1);YData_AR_Norm_T = Data_AR['TrueSD']
    #Data is normalized
    Scaled_Norm = preprocessing.StandardScaler().fit(XData_AR); XData_AR_Norm_T = Scaled_Norm.transform(XData_AR)
    #Model with FC layers is defined
    model = FC_Model(XData_AR_Norm_T.shape[1], DropoutRate=Dropout, LearningRate=LearningRate)
    model.fit(XData_AR_Norm_T, YData_AR_Norm_T, epochs=Epochs, verbose=0, batch_size=BatchSize); tf.keras.backend.clear_session()
    Forecast, Date_Forecast, TrainPrediction, ReturnForecast = ANN_ARCH_Forecast (Database, Lag, LagSD, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH,Scaled_Norm, XData_AR, model)
    VaR = ANN_ARCH_VaR(Alpha, Data['DailyReturnsOld'], Forecast,DF)
    return {'ANN_ARCH_model':model, 'Forecast_ANN_ARCH':Forecast, 'Date_Forecast':Date_Forecast, 'TrainPrediction': TrainPrediction, 'Scaler':Scaled_Norm, 'Forecast_GARCH':For_CV_GARCH, 'Forecast_GJR_GARCH':For_CV_GJR_GARCH, 'Forecast_TARCH':For_CV_TARCH, 'Forecast_EGARCH':For_CV_EGARCH, 'Forecast_AVGARCH':For_CV_AVGARCH, 'Forecast_FIGARCH':For_CV_FIGARCH, 'ReturnForecast':ReturnForecast, 'GARCH_fit': GARCH_Parameters, 'GJR_GARCH_fit':GJR_GARCH_Parameters, 'TARCH_fit':TARCH_Parameters, 'EGARCH_fit':EGARCH_Parameters, 'AVGARCH_fit':AVGARCH_Parameters, 'FIGARCH_fit':FIGARCH_Parameters, 'GARCH': GARCH, 'GJR_GARCH':GJR_GARCH, 'TARCH':TARCH, 'EGARCH':EGARCH, 'AVGARCH':AVGARCH, 'FIGARCH':FIGARCH, 'YData_Train':YData_AR_Norm_T, 'VaR': VaR}
#     return (model, Forecast, Date_Forecast, TrainPrediction, Scaled_Norm, For_CV_GARCH, For_CV_GJR_GARCH, For_CV_TARCH, For_CV_EGARCH, For_CV_AVGARCH, For_CV_FIGARCH, ReturnForecast, GARCH_Parameters, GJR_GARCH_Parameters, TARCH_Parameters, EGARCH_Parameters, AVGARCH_Parameters, FIGARCH_Parameters, GARCH, GJR_GARCH, TARCH, EGARCH, AVGARCH, FIGARCH, YData_AR_Norm_T)
